In [ ]:
import numpy as np
import pandas as pd
import csv
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, RepeatVector, Dense
from tensorflow.keras.layers import Reshape
from sklearn.preprocessing import MinMaxScaler

##Peaks Normalization and Log Transformation

In [ ]:
data = []
with open('CrestFactor.csv') as f:
    reader = csv.reader(f)
    next(reader) # skip header
    for row in reader:
        float_row = []
        for x in row[1:]:
            if x == '':
                float_row.append(np.nan)
            else:
                float_row.append(float(x))
        data.append(float_row)

# Replace NaN with 0
data = [np.nan_to_num(x) for x in data]

data=pd.DataFrame(data)

data = data.iloc[91:]
timesteps = 30
n_features = data.shape[1]

In [ ]:
data

,0,1,2,3,4,5,6,7,8,9,10
91,2.63469,4.062637,3.129854,4.609137,3.400028,3.178541,3.277437,2.873765,2.479360,2.627003,2.917039
92,3.23238,4.078335,7.042780,2.701289,2.655218,2.400014,3.628568,3.553896,2.173840,3.197476,3.343181
93,3.03576,4.810912,4.882066,2.596176,2.533021,2.524444,4.248105,2.962326,2.775282,3.781715,3.234808
94,3.29761,3.189906,2.945253,2.087754,3.301413,1.889706,3.933348,3.393899,2.747097,4.164268,3.000906
95,2.90987,4.485154,3.041485,2.553184,2.612140,2.383652,3.968591,3.508693,3.841308,2.683541,2.906957
96,2.86368,3.719087,6.612660,2.638301,2.869519,2.312029,4.383785,3.050037,2.950206,2.508664,2.461728
97,4.10185,4.826245,3.330945,2.588093,3.421568,2.400348,4.195410,2.644479,3.606444,2.992065,3.111706
98,5.03276,5.343038,6.252811,3.203273,2.773179,4.896499,3.477735,3.404237,2.914931,3.365790,2.738528
99,3.09773,3.994526,5.608631,2.984639,3.475719,2.932489,4.005264,3.464688,3.356123,3.345502,3.068255
100,2.97183,4.022532,3.628204,1.829227,3.635316,2.109542,2.781441,2.704160,2.429076,3.209185,3.446112


In [ ]:
inputs = Input(shape=(timesteps, 1))
x = LSTM(30)(inputs)
# x = Dense(10)(x)
# x = Dense(15)(x)
encoded = Dense(1)(x)

# decoded = Dense(16)(encoded)
# decoded = Dense(32)(decoded)
# decoded = Dense(61)(decoded)
# decoded = Reshape((61, 1))(decoded)
# decoded = LSTM(1, return_sequences=True)(decoded)
decoded = Dense(timesteps)(encoded)
decoded = Reshape((timesteps, 1))(decoded)
decoded = LSTM(30, return_sequences=True)(decoded)
# decoded = Dense(10)(decoded)
# decoded = Dense(15)(decoded)
decoded = Dense(1)(decoded)

latent_space_model = Model(inputs, encoded)
autoencoder = Model(inputs, decoded)
autoencoder.compile(optimizer='adam', loss='mae')


In [ ]:
autoencoder.summary()

Model: "model_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 30, 1)]           0         
                                                                 
 lstm_12 (LSTM)              (None, 30)                3840      
                                                                 
 dense_34 (Dense)            (None, 1)                 31        
                                                                 
 dense_35 (Dense)            (None, 30)                60        
                                                                 
 reshape_6 (Reshape)         (None, 30, 1)             0         
                                                                 
 lstm_13 (LSTM)              (None, 30, 30)            3840      
                                                                 
 dense_36 (Dense)            (None, 30, 1)             31 

In [ ]:
for i in range(n_features):
    print(f'Training on column: {i+1}/{n_features}')
    x_train = data.iloc[:, i].values.reshape(-1, timesteps, 1)

    autoencoder.fit(x_train, x_train, epochs=40, batch_size=8,verbose=0)

# Save model weights
autoencoder.save_weights('autoencoder_weights.h5')

Training on column: 1/11
Training on column: 2/11
Training on column: 3/11
Training on column: 4/11
Training on column: 5/11
Training on column: 6/11
Training on column: 7/11
Training on column: 8/11
Training on column: 9/11
Training on column: 10/11
Training on column: 11/11


In [ ]:
def predict_sequence_and_latent_space(input_sequence):
    input_sequence = np.array(input_sequence)
    input_sequence = input_sequence.reshape(1, -1, 1)
    print(input_sequence.shape)
    predicted_sequence = autoencoder.predict(input_sequence)
    latent_space_output = latent_space_model.predict(input_sequence)
    return predicted_sequence.reshape(-1), latent_space_output[0][0]

column_to_predict = 0
input_sequence = data.iloc[:, column_to_predict].values.tolist()
predictions, latent_space_output = predict_sequence_and_latent_space(input_sequence)

print("Original Sequence:")
print(input_sequence)

print("Predicted Sequence:")
print(predictions)

print("Latent Space Output:")
print(latent_space_output)

(1, 30, 1)
1/1 [==============================] - 0s 34ms/step
Original Sequence:
[2.63469, 3.23238, 3.03576, 3.29761, 2.90987, 2.86368, 4.10185, 5.03276, 3.09773, 2.97183, 3.16903, 3.10598, 3.58805, 2.87144, 3.13695, 3.02256, 3.12986, 2.73961, 3.01889, 4.67044, 3.06238, 2.68976, 2.96752, 2.911, 3.4472, 5.2182, 3.66436, 3.6346, 5.86073, 2.67534]
Predicted Sequence:
[2.8639705 3.3132727 3.1705136 2.99937   2.9033399 2.9292169 3.027222
 3.1219194 3.3274884 3.1514394 2.935743  2.9115427 2.7028275 2.9562721
 3.020638  3.064098  3.143869  2.9832447 2.959277  3.2803388 2.6832507
 2.7955947 2.7903392 2.8811479 2.943343  2.859746  3.0973077 3.1384845
 2.7583277 2.819826 ]
Latent Space Output:
11.878486


In [ ]:
import pandas as pd

# Convert the predicted sequence back to a DataFrame
predictions_df = pd.DataFrame(predictions, columns=['Predicted Sequence'])

# Get the original DataFrame
original_df = pd.DataFrame(input_sequence, columns = ['Original Sequence'])

# Get the column to predict (column_to_predict) and the corresponding column name
# column_name = original_df.columns[column_to_predict]

# Append the predicted sequence to the original DataFrame
original_df['Predicted Sequence'] = predictions_df

# Save the DataFrame to a CSV file
original_df.to_csv('predicted_sequences.csv', index=False)

print("Original Sequence and Predicted Sequence saved to 'predicted_sequences.csv'")


Original Sequence and Predicted Sequence saved to 'predicted_sequences.csv'


In [ ]:
num_columns = 11  # Total number of columns in your data

def predict_and_get_latent_output(input_sequence, column_idx):
    input_sequence = np.array(input_sequence)
    input_sequence = input_sequence.reshape(1, -1, 1)
    predicted_sequence = autoencoder.predict(input_sequence)
    latent_space_output = latent_space_model.predict(input_sequence)
    return predicted_sequence.reshape(-1), latent_space_output.reshape(-1)

# Create an array to store the latent_space_op values for each column
latent_space_op_array = []

# Loop over each column
for column_idx in range(num_columns):
    input_sequence = data.iloc[:, column_idx].values.tolist()
    predictions, latent_space_output = predict_and_get_latent_output(input_sequence, column_idx)
    latent_space_op_array.append(latent_space_output)

print(latent_space_op_array)

1/1 [==============================] - 0s 22ms/step
[array([11.878486], dtype=float32), array([11.8795595], dtype=float32), array([11.875491], dtype=float32), array([11.865106], dtype=float32), array([11.865122], dtype=float32), array([11.865057], dtype=float32), array([11.86418], dtype=float32), array([11.864109], dtype=float32), array([11.864088], dtype=float32), array([11.86503], dtype=float32), array([11.865132], dtype=float32)]


In [ ]:
latent_space_op_array

[array([11.878486], dtype=float32),
 array([11.8795595], dtype=float32),
 array([11.875491], dtype=float32),
 array([11.865106], dtype=float32),
 array([11.865122], dtype=float32),
 array([11.865057], dtype=float32),
 array([11.86418], dtype=float32),
 array([11.864109], dtype=float32),
 array([11.864088], dtype=float32),
 array([11.86503], dtype=float32),
 array([11.865132], dtype=float32)]

Data saved to output_data.csv


In [ ]:
data = [['Latent Space Output for each column:']]
for item in latent_space_op_array:
    data.append([item[0]])

# Write data to CSV file
csv_filename = 'latent_space_output.csv'
with open(csv_filename, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(data)
